# Introduction

Brett Wisniewski (+ the ISAT lab)

**Disclaimer**: PLEASE BE SURE TO WATCH THE VIDEO, VIEW THE README, READ THE PAPER, AND LOOK AT THE PREPROCESSING NOTEBOOK BEFORE THIS, THOSE WILL NOT MAKE A LOT OF SENSE WITHOUT IT.

All of this code collectively belongs to the research lab I am in. I played a part in writing it, but a lot of other people helped write it as well. I talked to Professor Fabio Abreu De Santos and he said it was fine if I used code that we all used since we are a research lab. Since I did this project with no 440 team, think of my research lab as my team in a way.


The code below isn't actually the exact code we used for our results. That code unfortunately requires a lot of time and resources to setup properly which is beyond the scope of this project. However, the code below does a very similar training methodology. Since the grader needs a way to run the code themselves, I have this other solution here as a more localized version.

In order to run this code, you will need to run it individually each time for each of the various ASR Transcription Modality combination. In the example we use Temporal_Google_Verbal.

BE SURE TO PROVIDE THE PROPER FILE PATH TO TEMPORAL_GOOGLE_VERBAL AND BE SURE TO HAVE AN OUTPUT FOLDER CREATED OR THE CODE WILL NOT WORK


We then have a series of import statements to run to make the code work:

In [13]:
from hyperopt import *
import pickle
import time
import numpy as np
import pandas as pd
import os
import csv
import statistics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GroupShuffleSplit
from sklearn.multioutput import MultiOutputClassifier


from sklearn.svm import LinearSVC

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneGroupOut

from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import GroupKFold

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In order to run this code, you will need to run it individually each time for each of the various ASR Transcriptn Modality combination. In the example we use Temporal_Google_Verbal. As you can see with the example below, all of the labels and bert embeddings are there (since we already handled that in the preprocessing step

In [14]:

pd.set_option('display.max_columns', None)
hunting_set_file = hunting_set_file = r"/content/Temporal_Google_Verbal.csv" # this is the path to our finished file
df_hunting = pd.read_csv(hunting_set_file)
df_hunting.dtypes

,0
utteranceID,object
Group,object
CPS_CONST_EstablishesCG_Confirms,float64
CPS_CONST_EstablishesCG_Interrupts,float64
CPS_CONST_SharesU_CorrectSolutions,float64
...,...
bert_embedding_764,float64
bert_embedding_765,float64
bert_embedding_766,float64
bert_embedding_767,float64


A visualization of the training file is provided below. As you can see, it has all of the BERT features ready to go (labels are still there but will be extracted in the next couple of cells)


In [15]:
df_hunting

utteranceID     Group  CPS_CONST_EstablishesCG_Confirms  \
0       Group_01_0  Group_01                               NaN   
1       Group_01_1  Group_01                               NaN   
2       Group_01_2  Group_01                               NaN   
3       Group_01_3  Group_01                               NaN   
4       Group_01_4  Group_01                               NaN   
...            ...       ...                               ...   
1817  Group_10_141  Group_10                               NaN   
1818  Group_10_142  Group_10                               NaN   
1819  Group_10_143  Group_10                               NaN   
1820  Group_10_144  Group_10                               NaN   
1821  Group_10_145  Group_10                               NaN   

      CPS_CONST_EstablishesCG_Interrupts  CPS_CONST_SharesU_CorrectSolutions  \
0                                    NaN                                 NaN   
1                                    NaN                                 NaN   
2                                    NaN                                 NaN   
3                                    NaN                                 NaN   
4                                    NaN                                 NaN   
...                                  ...                                 ...   
1817                                 NaN                                 NaN   
1818                                 NaN                                 NaN   
1819                                 NaN                                 NaN   
1820                                 NaN                                 NaN   
1821                                 NaN                                 NaN   

      CPS_CONST_SharesU_IncorrectSolutions  CPS_CONST_SharesU_Situation  \
0                                      NaN                          NaN   
1                                      NaN                          NaN   
2                                      NaN                          NaN   
3                                      NaN                          NaN   
4                                      NaN                          NaN   
...                                    ...                          ...   
1817                                   NaN                          NaN   
1818                                   NaN                          NaN   
1819                                   NaN                          NaN   
1820                                   NaN                          NaN   
1821                                   NaN                          NaN   

      CPS_MAINTAIN_FulfillsR_Apologizes  \
0                                   NaN   
1                                   NaN   
2                                   NaN   
3                                   NaN   
4                                   NaN   
...                                 ...   
1817                                NaN   
1818                                NaN   
1819                                NaN   
1820                                NaN   
1821                                NaN   

      CPS_MAINTAIN_FulfillsR_InitiatesOffTopic  \
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
...                                        ...   
1817                                       NaN   
1818                                       NaN   
1819                                       NaN   
1820                                       1.0   
1821                                       NaN   

      CPS_MAINTAIN_FulfillsR_JoinsOffTopic  CPS_MAINTAIN_FulfillsR_Support  \
0                                      NaN                             NaN   
1                                      NaN                             NaN   


In [16]:
#This is a visualization for the bert embeddings, to show all features are there
# It will also act as a conversion to numpy
X = df_hunting.iloc[:, 21:]
print(X.columns)
X = X.to_numpy()

Index(['bert_embedding_1', 'bert_embedding_2', 'bert_embedding_3',
       'bert_embedding_4', 'bert_embedding_5', 'bert_embedding_6',
       'bert_embedding_7', 'bert_embedding_8', 'bert_embedding_9',
       'bert_embedding_10',
       ...
       'bert_embedding_759', 'bert_embedding_760', 'bert_embedding_761',
       'bert_embedding_762', 'bert_embedding_763', 'bert_embedding_764',
       'bert_embedding_765', 'bert_embedding_766', 'bert_embedding_767',
       'bert_embedding_768'],
      dtype='object', length=768)


In [17]:
#Extracting the labels of the dataset into a new vector
y = df_hunting.iloc[:, 2:21]
print(y.columns)
y = y.to_numpy()


Index(['CPS_CONST_EstablishesCG_Confirms',
       'CPS_CONST_EstablishesCG_Interrupts',
       'CPS_CONST_SharesU_CorrectSolutions',
       'CPS_CONST_SharesU_IncorrectSolutions', 'CPS_CONST_SharesU_Situation',
       'CPS_MAINTAIN_FulfillsR_Apologizes',
       'CPS_MAINTAIN_FulfillsR_InitiatesOffTopic',
       'CPS_MAINTAIN_FulfillsR_JoinsOffTopic',
       'CPS_MAINTAIN_FulfillsR_Support', 'CPS_MAINTAIN_Initiative_Compliments',
       'CPS_MAINTAIN_Initiative_Criticizes',
       'CPS_MAINTAIN_Initiative_Suggestions', 'CPS_NEG_MonitorsE_GivingUp',
       'CPS_NEG_MonitorsE_Results', 'CPS_NEG_MonitorsE_Save',
       'CPS_NEG_MonitorsE_Strategizes', 'CPS_NEG_Responds_QuestionsOthers',
       'CPS_NEG_Responds_Reasons', 'CPS_NEG_Responds_Responds'],
      dtype='object')


Above is the label vector, as the labels had been extracted from the file

# Training / Testing

**Multi Label Classification Code:**

In [18]:
y_facets = []
for i in range(len(y)):
    const = 0
    neg = 0
    maintain = 0
    if(1 in y[i][:5]):
        const = 1
    if(1 in y[i][5:12]):
        neg = 1
    if(1 in y[i][12:]):
        maintain = 1
    y_facets.append([const,neg,maintain])
y = np.array(y_facets)
print(y[:18])

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]
 [0 0 0]
 [1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]]


The code above is designed for multi-label classification. To summarize, it basically creates a new list representing [const, neg, maintain], where const is set to 1 if the first segment contains 1, neg is set to 1 if the second segment contains 1, and maintain is set to 1 if the third segment contains 1.  

In [19]:
utteranceIDs = pd.DataFrame(df_hunting['utteranceID'], columns=['utteranceID'])
utteranceIDs = utteranceIDs.to_numpy()
utteranceIDs=utteranceIDs.reshape(-1)

groups = pd.DataFrame(df_hunting['Group'], columns=['Group'])
groups = groups.to_numpy()
groups = groups.reshape(-1)




Below we are defining the search space for various different machine learning models we will use for CPS classification.

In [20]:
#This will be the search space we use
space = hp.choice('classifier_type', [
    {
        'type': 'logistic_regression',
        'penalty' : hp.choice('logistic_regression_penalty', ['l2', 'none']),
        'solver' : hp.choice('logistic_regression_solver', ['lbfgs', 'newton-cg', 'sag', 'saga'])
    },
    {
        'type': 'LinSVC',
        'C': hp.uniform('LinSVC.C', 0, 3.0),

    },


    {
        'type': 'decision_tree',

        'criterion': hp.choice('decision_tree.criterion', ["gini", "entropy"]),
        'splitter': hp.choice('splitter', ["best", "random"]),
        'max_features': hp.choice('decision_tree.max_features', ["auto", "sqrt", "log2"]),

    },
    {
        'type': 'randomforest',
        'n_estimators': hp.choice('n_estimators', list(range(20, 170, 16))),
        'criterion': hp.choice('criterion', ["gini", "entropy"]),
        'max_features': hp.choice('max_features', [None, "sqrt", "log2"]),

    },
    {
        'type': 'knn',
        'n_neighbors': hp.choice('knn_n_neighbors', list(range(1, 150, 10)))
    }

])

The code below is about calculating statistics. The main one we are concerned with for this paper is the ROC

In [21]:
def f(params):
    global best, count, avg, all_results, final_score
    count += 1
    guide_metric, scores = hyperopt_train_test(params.copy(), groups, cv)
    results.append(scores)
    #writecsv here
    with open(os.path.join(modelcsv,scores[3]), 'w', newline = '') as modelCSV:
        writer = csv.writer(modelCSV)
        print("model: ", scores[3])
        writer.writerow(['utteranceID', 'True Label', 'Pred Label'])
        for pred in range(len(scores[26])):
            writer.writerow([utteranceIDs[pred], y[pred], scores[26][pred]])
    if guide_metric > best:
        print (scores)
        best = guide_metric
    else:
        print (scores)

    return {'loss': -guide_metric, 'status': STATUS_OK, 'eval_time': time.time(),
            'other_stuff': {'model':avg[0],'f1_0':avg[5] ,'acc_overall': avg[6],
                           'acc_balanced': avg[8],'kappa':avg[10] , 'roc': avg[12],'matthews_cc': avg[18]}
    }

The code beneath is the bulk of the training code.

In [22]:
def hyperopt_train_test(params, groups, cv):
    global avg
    global all_results
    global final_score
    model_name = params['type']
    predictioncsv = model_name
    del params['type']
    for item in params.values():
        predictioncsv += '_' + str(item)
    predictioncsv += '.csv'

    if model_name == 'naive_bayes':
        clf = GaussianNB(**params)
    elif model_name == 'AdaBoost':
        clf = AdaBoostClassifier(**params)
    elif model_name == 'LinSVC':
        clf = LinearSVC(**params)
    elif model_name == 'decision_tree':
        clf = DecisionTreeClassifier(**params, random_state=1) #(**params, n_jobs=6)
    elif model_name == 'randomforest':
        clf = RandomForestClassifier(**params, random_state=1) #(**params, n_jobs=6)
    elif model_name == 'knn':
        clf = KNeighborsClassifier(**params)
    elif model_name == 'logistic_regression':
        clf = LogisticRegression(**params, random_state=1)
    elif model_name == 'SVC':
        clf = SVC(**params, probability=True)
    else:
        return 0
    num_classes = 4
    if(is_binary): num_classes = 3
    if(is_binary): clf = MultiOutputClassifier(clf)
    y_pred = cross_val_predict(clf, X, y, groups=groups, n_jobs=20,cv=cv, verbose=0, method='predict')
    estimator = 'predict_proba'
    if model_name == 'LinSVC':
        y_score = []
        for num in range(num_classes):
            y_score.append(cross_val_predict(LinearSVC(**params), X, y[:,num], groups=groups, n_jobs=20,cv=cv, verbose=0, method='decision_function'))#'decision_function'
    else:
        y_score = cross_val_predict(clf, X, y, groups=groups, n_jobs=20,cv=cv, verbose=0, method=estimator)
    if(is_binary):
        f1_0 = f1_score(y, y_pred, average='weighted')
    else:
        f1_0 = f1_score(y, y_pred, labels=[0,1,2,3], average='weighted')
    print(f1_0)
    if(is_binary):
        cm = multilabel_confusion_matrix(y, y_pred)
    else:
        cm = confusion_matrix(y, y_pred)
    ###
    roc_auc_ovr = {}
    weighted_roc = {}
    matthews_cc = {}
    matthews_list = []
    roc_avg_list = []
    kappa_list = []
    acc_balanced_list = []
    acc_overall_list = []
    for class_id in range(num_classes):
        if(is_binary and model_name != "LinSVC"):
            prob_tmp = np.array(y_score)[class_id,:,1]
            print(prob_tmp.shape)
            true_max_tmp = y[:, class_id]
            pred_tmp = np.array(y_pred)[:, class_id]
        elif(is_binary and model_name == "LinSVC"):
            prob_tmp = np.array(y_score)[class_id,:]
            print(prob_tmp.shape)
            true_max_tmp = y[:, class_id]
            pred_tmp = np.array(y_pred)[:, class_id]
        else:
            prob_tmp = y_score[:, class_id]
            true_max_tmp = [1 if y_tmp == class_id else 0 for y_tmp in y]
            pred_tmp = [1 if p_tmp == class_id else 0 for p_tmp in y_pred]
        acc_balanced_list.append(balanced_accuracy_score(true_max_tmp, pred_tmp))
        acc_overall_list.append(accuracy_score(true_max_tmp, pred_tmp))
        try:
            roc_auc_ovr[class_id] = roc_auc_score(true_max_tmp, prob_tmp)
            weighted_roc[class_id] = roc_auc_score(true_max_tmp, prob_tmp, average='weighted')
            roc_avg_list.append(weighted_roc[class_id])
        except:
            print(f"ROC_AUC Issue with class {class_id}")
            roc_auc_ovr[class_id] = None
        try:
            kappa_list.append(cohen_kappa_score(true_max_tmp, pred_tmp))
            print('kappa: ', kappa_list)
        except:
            print(f"Kappa issue with class {class_id}")
        matthews_cc[class_id] = matthews_corrcoef(true_max_tmp, pred_tmp)
        matthews_list.append(matthews_cc[class_id])
    if(is_binary):
        matthews_cc[3] = 0
        roc_auc_ovr[3] = 0
    else:
        matthews_cc_3 = matthews_cc[3]
    roc_avg = sum(roc_avg_list)/num_classes
    roc_stdev = statistics.stdev(roc_avg_list)
    kappa = sum(kappa_list)/num_classes
    kappa_stdev = statistics.stdev(kappa_list)
    acc_balanced = sum(acc_balanced_list)/num_classes
    acc_balanced_stdev = statistics.stdev(acc_balanced_list)
    acc_overall = sum(acc_overall_list)/num_classes
    acc_overall_stdev = statistics.stdev(acc_overall_list)
    matthews_avg = sum(matthews_list)/num_classes
    matthews_stdev = statistics.stdev(matthews_list)
    ###

    #print("final_score", final_score)
    avg = [model_name, params, hunting_set_file, predictioncsv, is_binary,
           f1_0, acc_overall, acc_overall_stdev, acc_balanced, acc_balanced_stdev, kappa, kappa_stdev,
           roc_avg, roc_stdev, roc_auc_ovr[0], roc_auc_ovr[1], roc_auc_ovr[2], roc_auc_ovr[3],
           matthews_avg, matthews_stdev, matthews_cc[0], matthews_cc[1], matthews_cc[2], matthews_cc[3],
           cm, len(y_pred), y_pred]


    return avg[12], avg

Code below is for creating an output file

In [24]:
is_binary = True

participants = np.unique(groups)
print(participants)
len(participants)

output_csv = "440outputTest"
results_path = r"Output"
# Creat this locally manually in colab each time
best_results_csv = os.path.join(results_path, ".csv")
modelcsv = os.path.join(results_path, output_csv)
if not os.path.exists(modelcsv):
        os.mkdir(modelcsv)
else:
        print("WARNING: DIRECTORY ALREADY EXISTS")



['Group_01' 'Group_02' 'Group_03' 'Group_04' 'Group_05' 'Group_06'
 'Group_07' 'Group_08' 'Group_09' 'Group_10']


The code beneath here is for actually running the methods above as well as cross validation for each of the ten groups. You can run all of this code yourself, but it will take forever to actually train

In [25]:
avg = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]
results = []
group_kfold = GroupKFold(n_splits=len(participants))
logo = LeaveOneGroupOut()
cv = logo#.split(X, y, groups=groups) #group_kfold #

avg = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]
print (X.shape, y.shape)
count = 1
all_results =[]
best = 0

final_score = {
               'model_name':[],
               'hyper_paramaters':[],
               'f1_0':[],
               'acc_overall': [],
               'acc_balanced':[],
               'kappa':[],
               'roc':[],
               'roc_w':[],
               'cm': [],
               'roc_avg':[]}



# spark_trials = SparkTrials(parallelism=8)
best = fmin(
            f,
            space,
            algo=tpe.suggest,
            max_evals= 500,) # was 500, change as see fit depending on training
print ('best:' )
print(best)
print(space_eval(space, best))
df = pd.DataFrame(results, columns=["model_name", "params", "featurecsv", "predictioncsv", "is_binary",
                                    "f1", "acc_overall", "acc_overall_stdev", "acc_balanced", "acc_balanced_stdev", "kappa", "kappa_stdev",
                                    "roc_avg", "rov_stdev", 'roc_0', 'roc_1','roc_2', 'roc_3',
                                    "matthews_cc", "matthews_stdev", "matthews_0", "matthews_1", "matthews_2", "matthews_3",
                                    "cm", "num_samples", "y_pred"])
df = df.sort_values(by="roc_avg", ascending=False)
df.to_csv(os.path.join(modelcsv, output_csv) + ".csv")
best_model = list(df.iloc[0])
best_model.insert(0, output_csv)

with open (best_results_csv,'a',newline = '') as best_models:
    writer = csv.writer(best_models)
    writer.writerow(best_model)

(1822, 768) (1822, 3)
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



0.2856928905927736
(1822,)
kappa: 
[0.2202182790161772]
(1822,)
kappa: 
[0.2202182790161772, 0.0]
(1822,)
kappa: 
[0.2202182790161772, 0.0, 0.22088712588212078]
model: 
randomforest_entropy_sqrt_148.csv
['randomforest', {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 148}, '/content/Temporal_Google_Verbal.csv', 'randomforest_entropy_sqrt_148.csv', True, 0.2856928905927736, 0.7713135748261983, 0.06890239638593484, 0.56135842925876, 0.053144203130708854, 0.14703513496609932, 0.127336601277507, 0.750566581338466, 0.029227335792947583, 0.76404955376918, 0.7170315464895636, 0.7706186437566547, 0, 0.1670784944583729, 0.1447489294254921, 0.24663840920978317, 0.0, 0.25459707416533556, 0, array([[[1182,  105],
        [ 392,  143]],

       [[1550,    0],
        [ 272,    0]],

       [[1210,   87],
        [ 394,  131]]]), 1822, array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])]
100%|██████████| 1/1 [07:27

Results above show after runnng for one iteration a random forest model and how it performed